In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from pybaseball import statcast, cache
#from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import joblib
import math
from pybaseball import statcast
import scipy.stats as stats
#from catboost import Pool
#import optuna
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, precision_score
import sqlite3
cache.enable()

In [2]:
data = pd.read_csv('21-24pitching.csv')

In [3]:
data['release_speed'].mean()

np.float64(88.98224756397475)

In [4]:
data = data[~data['pitch_name'].isin(['Pitch Out', 'Eephus', 'Knuckleball'])]
data.loc[data['p_throws'] == 'L', 'pfx_x'] *= -1
data.loc[data['p_throws'] == 'L', 'release_pos_x'] *= -1
data.loc[data['p_throws'] == 'L', 'spin_axis'] = 360 - data.loc[data['p_throws'] == 'L', 'spin_axis']
fastballs = ['4-Seam Fastball', 'Sinker']
offspeed = ['Split-Finger', 'Changeup', 'Forkball']
breaking = ['Curveball', 'Slider', 'Cutter', 'Knuckle Curve', 'Sweeper', 'Screwball', 'Slow Curve']
print(data)

        pitch_type   game_date  release_speed  release_pos_x  release_pos_z  \
0               KC  2024-10-30           77.5          -1.11           5.65   
1               KC  2024-10-30           78.7          -1.01           5.73   
2               FC  2024-10-30           93.1          -1.19           5.53   
3               KC  2024-10-30           78.5          -1.19           5.70   
4               KC  2024-10-30           77.4          -1.23           5.78   
...            ...         ...            ...            ...            ...   
2990317         FF  2021-03-15           90.1          -2.72           6.10   
2990318         SI  2021-03-15           90.3          -2.71           6.05   
2990319         SI  2021-03-15           91.0          -2.93           6.07   
2990320         CH  2021-03-15           78.2          -2.45           6.17   
2990321         SL  2021-03-15           83.7          -2.48           6.17   

             player_name  batter  pitcher     event

In [5]:
def get_pitch_data(data, pitch_name, players):
    pitch_data = data.loc[(data['pitch_name'] == pitch_name) & (data['player_name'].isin(players))]
    grouped_data = pitch_data.groupby('player_name').agg({
        'release_speed': 'median',
        'pfx_x': 'median',
        'pfx_z': 'median'
    }).reset_index()
    
    return grouped_data

In [6]:
all = data['player_name'].unique()

# pitch type: Fastballs

fastball_data = get_pitch_data(data, '4-Seam Fastball', all)
fastball_data = fastball_data.dropna(subset=['release_speed'])
fastball_velo = dict(zip(fastball_data['player_name'], fastball_data['release_speed']))
fastball_horiz = dict(zip(fastball_data['player_name'], fastball_data['pfx_x']))
fastball_vert = dict(zip(fastball_data['player_name'], fastball_data['pfx_z']))

sinker_data = get_pitch_data(data, 'Sinker', all)
sinker_data = sinker_data.dropna(subset=['release_speed'])
sinker_velo = dict(zip(sinker_data['player_name'], sinker_data['release_speed']))
sinker_horiz = dict(zip(sinker_data['player_name'], sinker_data['pfx_x']))
sinker_vert = dict(zip(sinker_data['player_name'], sinker_data['pfx_z']))

cutter_data = get_pitch_data(data, 'Cutter', all)
cutter_data = cutter_data.dropna(subset=['release_speed'])
cutter_velo = dict(zip(cutter_data['player_name'], cutter_data['release_speed']))
cutter_horiz = dict(zip(cutter_data['player_name'], cutter_data['pfx_x']))
cutter_vert = dict(zip(cutter_data['player_name'], cutter_data['pfx_z']))

In [7]:
pitch_data = {
    '4-Seam Fastball': {
        'velo': fastball_velo,
        'horiz': fastball_horiz,
        'vert': fastball_vert
    },
    'Sinker': {
        'velo': sinker_velo,
        'horiz': sinker_horiz,
        'vert': sinker_vert
    },
    'Cutter': {
        'velo': cutter_velo,
        'horiz': cutter_horiz,
        'vert': cutter_vert
    }
}

def calculate_differences(row):
    player_name = row['player_name']
    for pitch_type in ['4-Seam Fastball', 'Sinker', 'Cutter']:
        if player_name in pitch_data[pitch_type]['velo']:
            velo_dif = row['release_speed'] - pitch_data[pitch_type]['velo'][player_name]
            horiz_dif = row['pfx_x'] - pitch_data[pitch_type]['horiz'][player_name]
            vert_dif = row['pfx_z'] - pitch_data[pitch_type]['vert'][player_name]
            return velo_dif, horiz_dif, vert_dif

    return np.nan, np.nan, np.nan

data['velo'], data['horiz_dif'], data['vert_dif'] = zip(*data.apply(calculate_differences, axis=1))

In [10]:
league_avg_run_exp = data['delta_run_exp'].mean()

clase = data[data['player_name'] == "Clase, Emmanuel"]
clase_cutter = clase[clase['pitch_name'] == 'Cutter']
clase_cutter_run_exp = clase_cutter['delta_run_exp'].mean()
clase_cutter_stuff = 100 + ((clase_cutter_run_exp/league_avg_run_exp) / 40)
print(clase_cutter_stuff)

burnes = data[data['player_name'] == "Burnes, Corbin"]
clase_cutter = burnes[burnes['pitch_name'] == 'Cutter']
clase_cutter_run_exp = clase_cutter['delta_run_exp'].mean()
clase_cutter_stuff = 100 + ((clase_cutter_run_exp/league_avg_run_exp) / 40)
print(clase_cutter_stuff)

119.76135442027109
115.02854400788944


In [12]:
# tester??
data_pitch_avg_rundiff_dict = {'4-Seam Fastball' : data[data['pitch_name'] == '4-Seam Fastball']['delta_run_exp'].mean(), 
'Sinker' : data[data['pitch_name'] == 'Sinker']['delta_run_exp'].mean(), 
'Split-Finger' : data[data['pitch_name'] == 'Split-Finger']['delta_run_exp'].mean(), 
'Changeup' : data[data['pitch_name'] == 'Changeup']['delta_run_exp'].mean(), 
'Forkball' : data[data['pitch_name'] == 'Forkball']['delta_run_exp'].mean(), 
'Curveball' : data[data['pitch_name'] == 'Curveball']['delta_run_exp'].mean(), 
'Slider' : data[data['pitch_name'] == 'Slider']['delta_run_exp'].mean(), 
'Cutter' : data[data['pitch_name'] == 'Cutter']['delta_run_exp'].mean(), 
'Knuckle Curve' : data[data['pitch_name'] == 'Knuckle Curve']['delta_run_exp'].mean(), 
'Sweeper' : data[data['pitch_name'] == 'Sweeper']['delta_run_exp'].mean(), 
'Screwball' : data[data['pitch_name'] == 'Screwball']['delta_run_exp'].mean(), 
'Slow Curve' : data[data['pitch_name'] == 'Slow Curve']['delta_run_exp'].mean(),
'Slurve' : data[data['pitch_name'] == 'Slurve']['delta_run_exp'].mean()}

stuff_list = []
for index, pitch in data.head(1000).iterrows():
    name_of_pitch = pitch['pitch_name']

    if (name_of_pitch in data_pitch_avg_rundiff_dict):
        pitch_mlb_delta = data_pitch_avg_rundiff_dict[name_of_pitch]
        stuff = 100 + ((pitch['delta_run_exp']/pitch_mlb_delta) / 20)
        stuff_list.append(stuff)

print(stuff_list)

[np.float64(110.47116321559074), np.float64(103.79389971579374), np.float64(101.66813148447521), np.float64(97.72366017052376), np.float64(131.86875761266745), np.float64(108.58552916845147), np.float64(100.0), np.float64(107.43604344295574), np.float64(104.67153792989271), np.float64(97.72366017052376), np.float64(103.49038773853025), np.float64(130.8064656922452), np.float64(104.54528014800371), np.float64(107.73955542021923), np.float64(111.8348375277215), np.float64(104.29276458422574), np.float64(108.65934174100278), np.float64(62.233692958942456), np.float64(110.9203779395829), np.float64(124.74556937614494), np.float64(98.5964415485376), np.float64(99.50361957204379), np.float64(106.25549306678202), np.float64(87.32220071798842), np.float64(112.12074706134325), np.float64(96.07989101148327), np.float64(104.67076815653058), np.float64(96.4135173083783), np.float64(53.41087848296189), np.float64(161.1087664342722), np.float64(104.08692213696426), np.float64(95.83349319766326), np.

In [13]:
league_avg_run_exp = data_pitch_avg_rundiff_dict['4-Seam Fastball']
print(league_avg_run_exp)


strider = data[data['player_name'] == "Strider, Spencer"]
Strider_fastball = strider[strider['pitch_name'] == '4-Seam Fastball']
Strider_fastball_exp = Strider_fastball['delta_run_exp'].mean()
Strider_fastball_stuff = 100 + ((Strider_fastball_exp/league_avg_run_exp) / 1)
print(Strider_fastball_stuff)

0.00039601519408887436
85.83590061287191


In [15]:
print(data_pitch_avg_rundiff_dict)
league_avg_run_exp = data['delta_run_exp'].mean()
print(league_avg_run_exp)

{'4-Seam Fastball': np.float64(0.00039601519408887436), 'Sinker': np.float64(-0.00018589186331006229), 'Split-Finger': np.float64(-0.002921146601146601), 'Changeup': np.float64(0.002395331958132783), 'Forkball': np.float64(-0.006101910828025479), 'Curveball': np.float64(0.000769204147189143), 'Slider': np.float64(-0.001594497812097812), 'Cutter': np.float64(-0.0005994731286512461), 'Knuckle Curve': np.float64(-0.0003294762892114243), 'Sweeper': np.float64(-0.0025752534854245877), 'Screwball': np.float64(0.031214007782101167), 'Slow Curve': np.float64(0.025886153846153847), 'Slurve': np.float64(-0.005104555064385932)}
-2.1138782766446598e-05


In [30]:
league_avg_run_exp = data_pitch_avg_rundiff_dict['4-Seam Fastball']
print(league_avg_run_exp)
#league_avg_run_exp = data['delta_run_exp'].mean()

cole = data[data['player_name'] == "Cole, Gerrit"]
cole_fastball = cole[cole['pitch_name'] == '4-Seam Fastball']
cole_fastball_exp = cole_fastball['delta_run_exp'].mean()
print(cole_fastball_exp)
cole_fastball_stuff = 100 + ((cole_fastball_exp/league_avg_run_exp) / 40)
print(cole_fastball_stuff)

0.00039601519408887436
-0.010448375574077226
99.34040563783694


In [31]:
sns.histplot(data['delta_run_exp'], kde=True, bins=50)
plt.title('Distribution of delta_run_exp')
plt.xlabel('Delta Run Expectancy')
plt.ylabel('Frequency')
plt.show()

NameError: name 'sns' is not defined

In [33]:
# Calculate league average delta_run_exp
delta_run_exp_avg = data['delta_run_exp'].mean()

# Transform predicted delta_run_exp into Stuff+ score
data['Stuff+'] = (data['delta_run_exp'] / delta_run_exp_avg) * 100

# Ensure MLB average pitch = 100
data['Stuff+'] = data['Stuff+'].apply(lambda x: max(50, min(x, 150)))  # Cap values between 50 and 150


In [42]:
cole = data[data['player_name'] == "Cole, Gerrit"]
cole['Stuff+'].mean()
clase = data[data['player_name'] == "Clase, Emmanuel"]
clase['Stuff+'].mean()
strider = data[data['player_name'] == "Strider, Spencer"]
strider['Stuff+'].mean()

np.float64(102.92229729729729)

In [48]:
swings_and_misses = data[data['description'].isin(['swinging_strike', 'swinging_strike_blocked'])]

In [56]:
stride_swm = swings_and_misses[swings_and_misses['player_name'] == "Strider, Spencer"]
stride_gen = data[data['player_name'] == "Strider, Spencer"]

In [57]:
len(stride_swm) / len(stride_gen)

0.17635135135135135

In [58]:
cole_swm = swings_and_misses[swings_and_misses['player_name'] == "Cole, Gerrit"]
cole_gen = data[data['player_name'] == "Cole, Gerrit"]
len(cole_swm) / len(cole_gen)

0.131678926521704

In [59]:
bum_swm = swings_and_misses[swings_and_misses['player_name'] == "Bumgarner, Madison"]
bum_gen = data[data['player_name'] == "Bumgarner, Madison"]
len(bum_swm) / len(bum_gen)

0.08343494164779655

In [60]:
burnes_swm = swings_and_misses[swings_and_misses['player_name'] == "Burnes, Corbin"]
burnes_gen = data[data['player_name'] == "Burnes, Corbin"]
len(burnes_swm) / len(burnes_gen)

0.1412892128757474